In [3]:
import os
os.chdir('..')

In [4]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [22]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pickle
from model.CVAE_all_chords import CVAE
from decode import *
import matplotlib.pyplot as plt
from utils import *

In [6]:
melody_framewise = np.load('./data/melody_data.npy')
chord_groundtruth_idx = np.load('./data/chord_groundtruth.npy')

melody = np.load('./data/melody_baseline.npy')
# chord = np.load('./data/number_96.npy')
# chord_onehot = np.load('./data/onehot_96.npy')
chord_onehot = np.load('./data/chord_onehot.npy')
length = np.load('./data/length.npy')

f = open('./data/tempos', 'rb')
tempos = pickle.load(f)
f.close()
f = open('./data/downbeats', 'rb')
downbeats = pickle.load(f)
f.close()

In [17]:
val_size = 500
print('splitting testing set...')
val_melody_framewise = melody_framewise[:val_size]
val_chord_groundtruth_idx = chord_groundtruth_idx[:val_size]

val_chord = torch.from_numpy(chord_onehot[:val_size]).float()
val_melody = torch.from_numpy(melody[:val_size]).float()
val_length = length[:val_size]



splitting testing set...


In [20]:
# Load model
device = 'cpu'
print('building model...')
with torch.no_grad():
    model = CVAE(device = device).to(device)
    model.load_state_dict(torch.load('output_models/model_cvae_all_chords.pth'))
    print(model)
    model.eval()
val_length, val_melody = val_length.to(device), val_melody.to(device)

building model...
CVAE(
  (encoder): LSTM(633, 256, num_layers=2, batch_first=True, bidirectional=True)
  (encoder_output2mean): Linear(in_features=512, out_features=16, bias=True)
  (encoder_output2logv): Linear(in_features=512, out_features=16, bias=True)
  (latent2decoder_input): Linear(in_features=592, out_features=128, bias=True)
  (decoder): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (outputs2chord): Linear(in_features=512, out_features=633, bias=True)
)


In [37]:
# Sampconductorne song
# torch.manual_seed(0)
song_index = 140
sample_melody = val_melody[song_index].unsqueeze(0)
sample_length = val_length[song_index].unsqueeze(0)
sample_chord = val_chord[song_index]
sample_melody_framewise = np.expand_dims(val_melody_framewise[song_index],axis=0)
sample_chord_groundtruth_idx = np.expand_dims(val_chord_groundtruth_idx[song_index],axis=0)
sample_tempo = [tempos[song_index]]
sample_downbeat = [downbeats[song_index]]

print(sample_length)

# bs = 1
# model.batch_size = bs
# torch.manual_seed(0)
latent_size = 16
z = torch.randn(1,272,latent_size)
z = torch.cat([z,sample_melody],dim=-1)

output_seq, chord_pred = model.decode(z)
print('chord pred shape',chord_pred[0].shape)

logits = torch.max(chord_pred[0][:sample_length[0]],-1)
gen_chord_index = logits.indices
print('max logits',logits)



tensor([8])
chord pred shape torch.Size([272, 633])
max logits torch.return_types.max(
values=tensor([0.2851, 0.3017, 0.1581, 0.1386, 0.1078, 0.1003, 0.1651, 0.3977],
       grad_fn=<MaxBackward0>),
indices=tensor([191, 191, 265, 265, 341, 234, 234,   0]))


In [38]:
########## Random sampling ###########
# Proceed chord decode
print('proceed chord decode...')
joint_prob = chord_pred.cpu().detach().numpy()

# Append argmax index to get pianoroll array
accompany_pianoroll = argmax2pianoroll_all(joint_prob)

# # augment chord into frame base
BEAT_RESOLUTION = 24
BEAT_PER_CHORD = 2

accompany_pianoroll_framewise, sample_chord_groundtruth_framewise = sequence2frame(accompany_pianoroll, sample_chord_groundtruth_idx)

# # length into frame base
sample_length_framewise = sample_length * BEAT_RESOLUTION * BEAT_PER_CHORD

# write pianoroll
result_dir = 'results/cvae_all_chords_test'
filename = str(song_index)
write_one_pianoroll(result_dir, filename, sample_melody_framewise, accompany_pianoroll_framewise, sample_chord_groundtruth_framewise, sample_length_framewise, sample_tempo, sample_downbeat)


proceed chord decode...
generate_pianoroll shape (1, 272, 128)
augment chord into frame base...
accompany_pianoroll frame shape: (1, 13056, 128)
groundtruth_pianoroll frame shape: (1, 13056, 128)
write pianoroll...
Finished!


In [39]:
joint_prob.shape

(1, 272, 633)